# Vignette

This vignette uses a simplified version of the NJ and VA Gubernatorial elections (from Feltham et al.) to demonstrate the analysis workflow for
TSCSMethods.

1. Load the packages, data:

In [1]:
import Pkg

Pkg.activate(".");
# Pkg.add("TSCSMethods"); Pkg.add("Dates")

using TSCSMethods, Dates

  Activating project at `~/Library/Mobile Documents/com~apple~CloudDocs/Yale/yale research/COVID19`


In [2]:
dat = example_data()

,date,fips,pop_dens,cumul_death_rate,death_rte,gub
,Date,Int64,Float64,Float64,Float64,Int64
1,2021-10-03,1001,92.8599,27.5362,0.0,0
2,2021-10-04,1001,92.8599,27.7174,0.0,0
3,2021-10-05,1001,92.8599,27.7174,0.0,0
4,2021-10-06,1001,92.8599,27.7174,0.181159,0
5,2021-10-07,1001,92.8599,27.7174,0.181159,0
6,2021-10-08,1001,92.8599,27.7174,0.0,0
7,2021-10-09,1001,92.8599,27.7174,0.0,0
8,2021-10-10,1001,92.8599,27.7174,0.0,0
9,2021-10-11,1001,92.8599,27.7174,0.0,0


Add an integer valued time variable:

In [3]:
dat[! ,:day] = Dates.value.(dat.date .- minimum(dat.date));

2. Construct the model object, and set the parameters:

In [4]:
matching_covariates = [:pop_dens, :cumul_death_rate];
timevary = Dict(:pop_dens => false, :cumul_death_rate => true);

model = makemodel(
  dat, :day, :fips, :gub, :death_rte,
  matching_covariates, timevary,
  10:20, -30:-1;
  title = "model",
  estimator = "ATT"
);

3. Perform matching:

In [5]:
match!(model, dat);

3. Perform balancing:

In [6]:
balance!(model, dat);

4. Estimate the ATTs for the non-refined, non-caliper model:

In [7]:
estimate!(model, dat);

5. Refine the model to the `refinementnum` (e.g., 5) best matches, and estimate:

In [16]:
refinedmodel = refine(
  model, dat;
  refinementnum = 5,
  dobalance = true, doestimate = true
);

6. Specify an initial caliper:
    You may choose to optionally specify a starting set of caliper values before running `autobalance()` which will successively apply calipers according to a simple algorithm, until sufficient balance is obtained on the matching covariates.

In [12]:
ibs = Dict(
  :cumul_death_rate => 0.25
);

Dict{Symbol, Float64} with 1 entry:
  :cumul_death_rate => 0.25

7. Successively apply calipers to get a refined caliper model with sufficient balance:

In [13]:
calmodel, refcalmodel, overall = autobalance(
  model, dat;
  calmin = 0.08, step = 0.05,
  initial_bals = ibs,
  dooverall = true
);

8. Save a record of the model, including the overall mean estimate across $F$:

In [18]:
savepath = pwd();

recordset = makerecords(
  dat, savepath, [model, refinedmodel, calmodel, refcalmodel]
)

TSCSMethods.save_object(savepath * "overall_estimate.jld2", overall)

9. Inspect the model results (e.g., for the refined caliper model):

In [19]:
refcalmodel.results

,f,att,mean,2.5%,50.0%,97.5%,treated,matches
,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,10,-0.154662,-0.154884,-0.234006,-0.153219,-0.0863879,145,705
2,11,-0.153194,-0.154948,-0.238823,-0.154008,-0.081035,145,705
3,12,-0.146457,-0.147949,-0.230159,-0.147754,-0.0715198,145,705
4,13,-0.047049,-0.0470613,-0.142212,-0.0482126,0.0469865,145,705
5,14,-0.151674,-0.152964,-0.244166,-0.152904,-0.0716866,145,705
6,15,-0.17604,-0.178863,-0.266755,-0.177122,-0.100019,145,705
7,16,-0.143117,-0.144497,-0.22003,-0.143469,-0.0778145,145,705
8,17,-0.187231,-0.18888,-0.272821,-0.187667,-0.113924,145,705
9,18,-0.148982,-0.150535,-0.235312,-0.149395,-0.0776914,145,705
